# Advanced Indexing

## Learning Objectives

* Orthogonal vs. Pointwise (Vectorized) Indexing.
* Pointwise indexing in Xarray to extract data at a collection of points.
* Understand the difference between NumPy and Xarray indexing behavior.

## Overview

In the previous notebooks, we learned basic forms of indexing with Xarray, including positional and label-based indexing, datetime indexing, and nearest neighbor lookups. We also learned that indexing an Xarray DataArray directly works (mostly) like it does for NumPy arrays; however, Xarray indexing behavior deviates from NumPy when using multiple arrays for indexing, like `arr[[0, 1], [0, 1]]`.

To better understand this difference, let's take a look at an example of 2D 5x5 array:

In [ ]:
import numpy as np

# Create a 5x5 array with values from 1 to 25
np_array = np.arange(1, 26).reshape(5, 5)
np_array

Now create a Xarray DataArray from this NumPy array: 

In [ ]:
import xarray as xr

da = xr.DataArray(np_array, dims=["x", "y"])
da

Now, let's see how the indexing behavior is different between NumPy array and Xarray DataArray when indexing with multiple arrays:

In [ ]:
np_array[[0, 2, 4], [0, 2, 4]]

In [ ]:
da[[0, 2, 4], [0, 2, 4]]

The image below summarizes the difference between vectorized and orthogonal indexing for a 2D 5x5 NumPy array and Xarray DataArray:



![Orthogonal vs. Vectorized Indexing](../../images/orthogonal_vs_vectorized.png)

**Pointwise** or **Vectorized indexing**, shown on the left, selects specific elements at given coordinates, resulting in an array of those individual elements. In the example shown, the indices `[0, 2, 4]`, `[0, 2, 4]` select the elements at positions `(0, 0)`, `(2, 2)`, and `(4, 4)`, resulting in the values `[1, 13, 25]`. This is the default behavior of NumPy arrays.
  
In contrast, **orthogonal indexing** uses the same indices to select entire rows and columns, forming the Cartesian product of the specified indices. This method results in sub-arrays that include all combinations of the selected rows and columns. The example demonstrates this by selecting rows 0, 2, and 4 and columns 0, 2, and 4, resulting in a subarray containing `[[1, 3, 5], [11, 13, 15], [21, 23, 25]]`. This is Xarray DataArray's default behavior.
 
The output of vectorized indexing is a `1D array`,  while the output of orthogonal indexing is a `3x3` array.   


:::{tip}  To Summarize: 

- *Pointwise* or *vectorized* indexing is a more general form of indexing that allows for arbitrary combinations of indexing arrays. This method of indexing is analogous to the broadcasting rules in NumPy, where the dimensions of the indexers are aligned and the result is determined by the shape of the indexers. This is the default behavior in NumPy.

- *Orthogonal* or *outer* indexing allows for indexing along each dimension independently, treating the indexers as one-dimensional arrays. The principle of outer or orthogonal indexing is that the result mirrors the effect of independently indexing along each dimension with integer or boolean arrays, treating both the indexed and indexing arrays as one-dimensional. This method of indexing is analogous to vector indexing in programming languages like MATLAB, Fortran, and R, where each indexer component independently selects along its corresponding dimension. This is the default behavior in Xarray.


:::

:::{admonition} Orthogonal indexing with NumPy
:class: note dropdown

While pointwise indexing is the default behavior in NumPy, you can achieve orthogonal indexing by using the [`np.ix_` function](https://numpy.org/doc/stable/reference/generated/numpy.ix_.html). This function constructs an open mesh from multiple arrays, allowing you to index along each dimension independently similar to Xarray indexing behavior. For example: 

```python
ixgrid = np.ix_([0, 2, 4], [0, 2, 4])
np_array[ixgrid]
```

:::

## Orthogonal Indexing in Xarray

As explained earlier, when you use only integers, slices, or unlabeled arrays (arrays without dimension names, such as `np.ndarray` or `list`, but not `DataArray`) to index an `Xarray DataArray`, Xarray interprets these indexers orthogonally. This means it indexes along independent axes, rather than using NumPy's broadcasting rules to vectorize the indexers. 

In the example above we saw this behavior, but let's see this behavior in action with a real dataset. Here we’ll use `air temperature` data from the National Center for Environmental Prediction:

In [ ]:
import numpy as np
import xarray as xr


xr.set_options(display_expand_attrs=False)
np.set_printoptions(threshold=10, edgeitems=2)
%config InlineBackend.figure_format='retina'

ds = xr.tutorial.load_dataset("air_temperature")
da_air = ds.air
da_air

In [ ]:
selected_da = da_air.isel(time=0, lat=[2, 4, 10, 13], lon=[1, 6, 7])  # -- orthogonal indexing
selected_da

👆 Please note that the output shape in the example above is `4x3` because the latitude indexer selects 4 rows, and the longitude indexer selects 3 columns.

For more flexibility, you can supply `DataArray()` objects as indexers. Dimensions on resultant arrays are given by the ordered union of the indexers’ dimensions.

For example, in the example below we do orthogonal indexing using `DataArray()` objects. 

In [ ]:
target_lat = xr.DataArray([31, 41, 42, 42], dims="degrees_north")
target_lon = xr.DataArray([200, 201, 202, 205], dims="degrees_east")

da_air.sel(lat=target_lat, lon=target_lon, method="nearest")  # -- orthogonal indexing

In the above example, you can see how the output shape is `time` x `lats` x `lons`. Please note that there are no shared dimensions between the indexers, so the output shape is the union of the dimensions of the indexers.

```{attention}
Please note that slices or sequences/arrays without named-dimensions are treated as if they have the same dimension which is indexed along.
```

For example:



In [ ]:
da_air.sel(lat=[20, 30, 40], lon=target_lon, method="nearest")


But what if we'd like to find the nearest climate model grid cell to a collection of specified points (for example observation sites, or weather stations)?

## Vectorized or Pointwise Indexing in Xarray

Like NumPy and pandas, Xarray supports indexing many array elements at once in a *vectorized* manner. 

**Vectorized indexing** or **Pointwise Indexing** using `DataArrays()` can be used to extract information from the nearest grid cells of interest, for example, the nearest climate model grid cells to a collection of specified observation tower data latitudes and longitudes.

```{hint}
To trigger vectorized indexing behavior, you will need to provide the selection dimensions with a new **shared** output dimension name. This means that the dimensions of both indexers must be the same, and the output will have the same dimension name as the indexers.
```

Let's see how this works with an example:

A researcher wants to find the nearest climate model grid cell to a collection of observation sites. They have the latitude and longitude of the observation sites as following:

In [ ]:
obs_lats = [31.81, 41.26, 22.59, 44.47, 28.57]

obs_lons = [200.16, 201.57, 305.54, 210.56, 226.59]

If the researcher use the lists to index the DataArray, they will get the orthogonal indexing behavior, which is not what they want.

In [ ]:
da_air.sel(lat=obs_lats, lon=obs_lats, method="nearest")  # -- orthogonal indexing

To trigger the pointwise indexing, they need to create DataArray objects with the same dimension name, and then use them to index the DataArray. 
For example, the code below first create DataArray objects for the latitude and longitude of the observation sites using a shared dimension name `points`, and then use them to index the DataArray `air_temperature`:

In [ ]:
## latitudes of weather stations with a dimension of "points"
lat_points = xr.DataArray(obs_lats, dims="points")
lat_points

In [ ]:
## longitudes of weather stations with a dimension of "points"
lon_points = xr.DataArray(obs_lons, dims="points")
lon_points

Now, retrieve data at the grid cells nearest to the target latitudes and longitudes (weather stations):

In [ ]:
da_air.sel(lat=lat_points, lon=lon_points, method="nearest")  # -- pointwise indexing

👆 Please notice how the shape of our `DataArray` is `time` x `points`, extracting time series for each weather stations. 


In [ ]:
da_air.sel(lat=lat_points, lon=lon_points, method="nearest").dims

Now, let's plot the data for all stations.

In [ ]:
da_air.sel(lat=lat_points, lon=lon_points, method="nearest").plot(x='time', hue='points');

## Exercises

::::{admonition} Exercise
:class: tip

In the simple 2D 5x5 Xarray data array above, select the sub-array containing (0,0),(2,2),(4,4):

:::{admonition} Solution
:class: dropdown
```python

indices = np.array([0, 2, 4])

index_da = xr.DataArray(indices, dims="points")

subset_da = da.sel(x=index_da, y=index_da)
subset_da
```
:::
::::

## Additional Resources

- [Xarray Docs - Indexing and Selecting Data](https://docs.xarray.dev/en/stable/indexing.html)


:::{seealso}
- [Introductions to Fancy Indexing](https://jakevdp.github.io/PythonDataScienceHandbook/02.07-fancy-indexing.html)
- [NumPy Docs - Advanced Indexing](https://numpy.org/doc/stable/user/basics.indexing.html#advanced-indexing)

:::
